In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as checkpoint

# RepVGG

<img src="https://i.ibb.co/wrxq7Kv/image.png" alt="image" border="0">

## Reparameterization

Chứng minh kỹ thuật reparameterization, gọi đầu vào hình ảnh $X \in \mathbb{R}^{n \times n}$ là một ma trận có dạng:

$$ X = 
\begin{pmatrix}
x_{11} & x_{12} & x_{13} & x_{14} & x_{15} \\
x_{21} & x_{22} & x_{23} & x_{24} & x_{25} \\ 
x_{31} & x_{32} & x_{33} & x_{34} & x_{35} \\
x_{41} & x_{42} & x_{44} & x_{44} & x_{45} \\
x_{51} & x_{52} & x_{55} & x_{55} & x_{55} \\
\end{pmatrix}
$$

và ma trận trọng số $W$ thực hiện convolution:

$$ W = 
\begin{pmatrix}
w_{11} & w_{12} & w_{13} \\
w_{21} & w_{22} & w_{23} \\
w_{31} & w_{32} & w_{33} \\
\end{pmatrix}
$$

Gọi ma trận kết quả sau khi thực hiện phép convolution (padding=$1$) $X * W$ là $Y$:

$$ Y =
\begin{pmatrix}
y_{11} & y_{12} & y_{13} & y_{14} & y_{15} \\
y_{21} & y_{22} & y_{23} & y_{24} & y_{25} \\ 
y_{31} & y_{32} & y_{33} & y_{34} & y_{35} \\
y_{41} & y_{42} & y_{44} & y_{44} & y_{45} \\
y_{51} & y_{52} & y_{55} & y_{55} & y_{55} \\
\end{pmatrix}
$$

Trong đó $y_{22} = x_{11} \times w_{11} + x_{12} \times w_{12} + \dots + x_{33} \times w_{33}$

Gọi ma trận kết quả sau khi thực hiện phép convolution pointwise với giá trị $p$ là $Z$:
$$ Z =
\begin{pmatrix}
z_{11} & z_{12} & z_{13} & z_{14} & z_{15} \\
z_{21} & z_{22} & z_{23} & z_{24} & z_{25} \\ 
z_{31} & z_{32} & z_{33} & z_{34} & z_{35} \\
z_{41} & z_{42} & z_{44} & z_{44} & z_{45} \\
z_{51} & z_{52} & z_{55} & z_{55} & z_{55} \\
\end{pmatrix}
$$

Trong đó $z_{22} = x_{22} \times p$
Vậy trong ma trận kết quả $K = Y + Z$ thì vị trí $K_{22} = y_{22} + z_{22} = (x_{11} \times w_{11} + x_{12} \times w_{12} + \dots + x_{33} \times w_{33}) + (x_{22} \times p)$


Đây là operations diễn ra trong lúc train. Tuy nhiên khi inference thì ta áp dụng kỹ thuật reparameterization để giảm lượng params tính toán hay FLOPS bằng các bước sau:
1. Padding giá trị $p$ thành ma trận trọng số $P$ có shape bằng ma trận $W$
2. Tạo ma trận trọng số mởi là $E = W + E$
3. Thực hiện phép convolution $X * E$ 

Lúc này ma trận được padded $P$ có dạng
$$ P = 
\begin{pmatrix}
0 & 0 & 0 \\
0 & p & 0 \\
0 & 0 & 0 \\
\end{pmatrix}
$$

Ma trận trọng số mới $E$ có dạng:
$$ E = 
\begin{pmatrix}
w_{11} & w_{12} & w_{13} \\
w_{21} & w_{22} + p & w_{23} \\
w_{31} & w_{32} & w_{33} \\
\end{pmatrix}
$$

Ma trận kết quả $K'$ có thành phần $K'_{22} = x_{11} \times w_{11} + x_{12} \times w_{12} + \dots + (w_{22} + p) \times x_{22} + \dots + x_{33} \times w_{33} $. \
Chú ý phần tử $(w_{22} + p) \times x_{22} = w_{22} \times x_{22} + x_{22} \times p$
Từ đây ta dễ dàng so sánh và thấy $K = K'$

In [ ]:
X = torch.tensor([
    [1, 2, 3, 4, 5],
    [2, 3, 5, 1, 2],
    [9, 8, 1, 5, 4],
    [0, 1, 2, 4, 0],
    [5, 6, 7, 8, 9],
]).unsqueeze(0).to(torch.float32)

W = torch.tensor([
    [0.5, 0.1, 0.2],
    [0.2, 0.3, 0.9],
    [0.1, 0.4, 0.6],
]).unsqueeze(0).unsqueeze(0)

P = torch.tensor([
    [0.7]
]).unsqueeze(0).unsqueeze(0)

Y = F.conv2d(X, W, stride=1, padding=1)
Z = F.conv2d(X, P, stride=1, padding=0)
K = Y + Z
K

tensor([[[ 5.4000,  9.3000,  9.9000, 11.2000,  6.7000],
         [13.6000, 13.9000, 12.8000, 11.2000,  6.8000],
         [17.6000, 14.6000, 12.6000, 13.6000,  6.1000],
         [ 9.0000, 15.4000, 19.1000, 15.5000,  8.1000],
         [10.6000, 13.8000, 16.9000, 18.9000, 12.6000]]])

In [ ]:
X = torch.tensor([
    [1, 2, 3, 4, 5],
    [2, 3, 5, 1, 2],
    [9, 8, 1, 5, 4],
    [0, 1, 2, 4, 0],
    [5, 6, 7, 8, 9],
]).unsqueeze(0).to(torch.float32)

W = torch.tensor([
    [0.5, 0.1, 0.2],
    [0.2, 0.3, 0.9],
    [0.1, 0.4, 0.6],
]).unsqueeze(0).unsqueeze(0)

P = torch.tensor([
    [0., 0., 0.],
    [0., 0.7, 0.],
    [0., 0., 0.],
]).unsqueeze(0).unsqueeze(0)

E = W + P

K_ = F.conv2d(X, E, stride=1, padding=1)
K_

tensor([[[ 5.4000,  9.3000,  9.9000, 11.2000,  6.7000],
         [13.6000, 13.9000, 12.8000, 11.2000,  6.8000],
         [17.6000, 14.6000, 12.6000, 13.6000,  6.1000],
         [ 9.0000, 15.4000, 19.1000, 15.5000,  8.1000],
         [10.6000, 13.8000, 16.9000, 18.9000, 12.6000]]])

## Reparam BatchNorm

Cho một ma trận $M \in R^{n \times n}$, thì $\text{BN}(M)$ được tính bằng cách:

- Tính giá trị trung bình $\mu$:
$$ \mu = \frac{1}{m} \sum_{i=1}^m M_i $$

- Sau đó tính độ bình phương độ lệch chuẩn $\sigma^2$:
$$ \sigma^2 = \frac{1}{m} \sum_{i=1}^m (M_i - \mu)^2 $$

- Sau đó thực hiện normalization:
$$ \hat{M} = \frac{M - \mu}{\sqrt{\sigma^2 + \epsilon}} $$

- Sau đó ta có thể Scale (biến đổi) và Shift/Transalte (dịch chuyển) ma trận này bằng cách:
$$ bn(M, \mu, \sigma, \gamma, \beta) = \gamma \hat{M} + \beta $$ 

$$ bn(M, \mu, \sigma, \gamma, \beta) = \gamma \frac{M - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta $$

với $\mu, \sigma, \gamma, \beta$ là các giá trị cho trước. Giả sử $\epsilon=0$ thì ta có thể viết đơn giản lại thành:

$$ bn(M, \mu, \sigma, \gamma, \beta) = (M - \mu) \frac{\gamma}{\sigma} + \beta $$

Vậy nếu ta lấy Batch Norm của ma trận là kết quả phép convolution $M * W$ thì có dạng:

$$ bn(M * W, \mu, \sigma, \gamma, \beta) = (M * W - \mu) \frac{\gamma}{\sigma} + \beta $$
$$= M * (W \frac{\gamma}{\sigma}) + (- \mu \frac{\gamma}{\sigma} + \beta) $$

Biểu thức sẽ cho ta ma trận trọng số mới $W' = W \frac{\gamma}{\sigma}$ và bias mới $b = - \mu \frac{\gamma}{\sigma} + \beta$

### How BatchNorm works

## Torchvision version

In [ ]:
def conv_bn(in_channels, out_channels, kernel_size, stride, padding, groups=1):
    result = nn.Sequential()
    result.add_module('conv', nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                                  kernel_size=kernel_size, stride=stride, padding=padding, groups=groups, bias=False))
    result.add_module('bn', nn.BatchNorm2d(num_features=out_channels))
    return result

class RepVGGBlock(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=0, dilation=1, groups=1, padding_mode='zeros', deploy=False, use_se=False):
        super(RepVGGBlock, self).__init__()
        self.deploy = deploy
        self.groups = groups
        self.in_channels = in_channels

        assert kernel_size == 3
        assert padding == 1

        padding_11 = padding - kernel_size // 2

        self.nonlinearity = nn.ReLU()

        if use_se:
            #   Note that RepVGG-D2se uses SE before nonlinearity. But RepVGGplus models uses SE after nonlinearity.
            self.se = SEBlock(out_channels, internal_neurons=out_channels // 16)
        else:
            self.se = nn.Identity()

        if deploy:
            self.rbr_reparam = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride,
                                      padding=padding, dilation=dilation, groups=groups, bias=True, padding_mode=padding_mode)

        else:
            self.rbr_identity = nn.BatchNorm2d(num_features=in_channels) if out_channels == in_channels and stride == 1 else None
            self.rbr_dense = conv_bn(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, groups=groups)
            self.rbr_1x1 = conv_bn(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=stride, padding=padding_11, groups=groups)
            print('RepVGG Block, identity = ', self.rbr_identity)


    def forward(self, inputs):
        if hasattr(self, 'rbr_reparam'):
            return self.nonlinearity(self.se(self.rbr_reparam(inputs)))

        if self.rbr_identity is None:
            id_out = 0
        else:
            id_out = self.rbr_identity(inputs)

        return self.nonlinearity(self.se(self.rbr_dense(inputs) + self.rbr_1x1(inputs) + id_out))


    #   Optional. This may improve the accuracy and facilitates quantization in some cases.
    #   1.  Cancel the original weight decay on rbr_dense.conv.weight and rbr_1x1.conv.weight.
    #   2.  Use like this.
    #       loss = criterion(....)
    #       for every RepVGGBlock blk:
    #           loss += weight_decay_coefficient * 0.5 * blk.get_cust_L2()
    #       optimizer.zero_grad()
    #       loss.backward()
    def get_custom_L2(self):
        K3 = self.rbr_dense.conv.weight
        K1 = self.rbr_1x1.conv.weight
        t3 = (self.rbr_dense.bn.weight / ((self.rbr_dense.bn.running_var + self.rbr_dense.bn.eps).sqrt())).reshape(-1, 1, 1, 1).detach()
        t1 = (self.rbr_1x1.bn.weight / ((self.rbr_1x1.bn.running_var + self.rbr_1x1.bn.eps).sqrt())).reshape(-1, 1, 1, 1).detach()

        l2_loss_circle = (K3 ** 2).sum() - (K3[:, :, 1:2, 1:2] ** 2).sum()      # The L2 loss of the "circle" of weights in 3x3 kernel. Use regular L2 on them.
        eq_kernel = K3[:, :, 1:2, 1:2] * t3 + K1 * t1                           # The equivalent resultant central point of 3x3 kernel.
        l2_loss_eq_kernel = (eq_kernel ** 2 / (t3 ** 2 + t1 ** 2)).sum()        # Normalize for an L2 coefficient comparable to regular L2.
        return l2_loss_eq_kernel + l2_loss_circle



#   This func derives the equivalent kernel and bias in a DIFFERENTIABLE way.
#   You can get the equivalent kernel and bias at any time and do whatever you want,
    #   for example, apply some penalties or constraints during training, just like you do to the other models.
#   May be useful for quantization or pruning.
    def get_equivalent_kernel_bias(self):
        kernel3x3, bias3x3 = self._fuse_bn_tensor(self.rbr_dense)
        kernel1x1, bias1x1 = self._fuse_bn_tensor(self.rbr_1x1)
        kernelid, biasid = self._fuse_bn_tensor(self.rbr_identity)
        return kernel3x3 + self._pad_1x1_to_3x3_tensor(kernel1x1) + kernelid, bias3x3 + bias1x1 + biasid

    def _pad_1x1_to_3x3_tensor(self, kernel1x1):
        if kernel1x1 is None:
            return 0
        else:
            return torch.nn.functional.pad(kernel1x1, [1,1,1,1])

    def _fuse_bn_tensor(self, branch):
        if branch is None:
            return 0, 0
        if isinstance(branch, nn.Sequential):
            kernel = branch.conv.weight
            running_mean = branch.bn.running_mean
            running_var = branch.bn.running_var
            gamma = branch.bn.weight
            beta = branch.bn.bias
            eps = branch.bn.eps
        else:
            assert isinstance(branch, nn.BatchNorm2d)
            if not hasattr(self, 'id_tensor'):
                input_dim = self.in_channels // self.groups
                kernel_value = np.zeros((self.in_channels, input_dim, 3, 3), dtype=np.float32)
                for i in range(self.in_channels):
                    kernel_value[i, i % input_dim, 1, 1] = 1
                self.id_tensor = torch.from_numpy(kernel_value).to(branch.weight.device)
            kernel = self.id_tensor
            running_mean = branch.running_mean
            running_var = branch.running_var
            gamma = branch.weight
            beta = branch.bias
            eps = branch.eps
        std = (running_var + eps).sqrt()
        t = (gamma / std).reshape(-1, 1, 1, 1)
        return kernel * t, beta - running_mean * gamma / std

    def switch_to_deploy(self):
        if hasattr(self, 'rbr_reparam'):
            return
        kernel, bias = self.get_equivalent_kernel_bias()
        self.rbr_reparam = nn.Conv2d(in_channels=self.rbr_dense.conv.in_channels, out_channels=self.rbr_dense.conv.out_channels,
                                     kernel_size=self.rbr_dense.conv.kernel_size, stride=self.rbr_dense.conv.stride,
                                     padding=self.rbr_dense.conv.padding, dilation=self.rbr_dense.conv.dilation, groups=self.rbr_dense.conv.groups, bias=True)
        self.rbr_reparam.weight.data = kernel
        self.rbr_reparam.bias.data = bias
        self.__delattr__('rbr_dense')
        self.__delattr__('rbr_1x1')
        if hasattr(self, 'rbr_identity'):
            self.__delattr__('rbr_identity')
        if hasattr(self, 'id_tensor'):
            self.__delattr__('id_tensor')
        self.deploy = True



class RepVGG(nn.Module):

    def __init__(self, num_blocks, num_classes=1000, width_multiplier=None, override_groups_map=None, deploy=False, use_se=False, use_checkpoint=False):
        super(RepVGG, self).__init__()
        assert len(width_multiplier) == 4
        self.deploy = deploy
        self.override_groups_map = override_groups_map or dict()
        assert 0 not in self.override_groups_map
        self.use_se = use_se
        self.use_checkpoint = use_checkpoint

        self.in_planes = min(64, int(64 * width_multiplier[0]))
        self.stage0 = RepVGGBlock(in_channels=3, out_channels=self.in_planes, kernel_size=3, stride=2, padding=1, deploy=self.deploy, use_se=self.use_se)
        self.cur_layer_idx = 1
        self.stage1 = self._make_stage(int(64 * width_multiplier[0]), num_blocks[0], stride=2)
        self.stage2 = self._make_stage(int(128 * width_multiplier[1]), num_blocks[1], stride=2)
        self.stage3 = self._make_stage(int(256 * width_multiplier[2]), num_blocks[2], stride=2)
        self.stage4 = self._make_stage(int(512 * width_multiplier[3]), num_blocks[3], stride=2)
        self.gap = nn.AdaptiveAvgPool2d(output_size=1)
        self.linear = nn.Linear(int(512 * width_multiplier[3]), num_classes)

    def _make_stage(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        blocks = []
        for stride in strides:
            cur_groups = self.override_groups_map.get(self.cur_layer_idx, 1)
            blocks.append(RepVGGBlock(in_channels=self.in_planes, out_channels=planes, kernel_size=3,
                                      stride=stride, padding=1, groups=cur_groups, deploy=self.deploy, use_se=self.use_se))
            self.in_planes = planes
            self.cur_layer_idx += 1
        return nn.ModuleList(blocks)

    def forward(self, x):
        out = self.stage0(x)
        for stage in (self.stage1, self.stage2, self.stage3, self.stage4):
            for block in stage:
                if self.use_checkpoint:
                    out = checkpoint.checkpoint(block, out)
                else:
                    out = block(out)
        out = self.gap(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out